In [1]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)
/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


# Exploring the relationship between low/high tapasin expressers and HLA genotype <br>
Hypothesis: Since the functional relationship between these genes is important and they come from the same region of the genome, we think there may be a germline relationship <br>
Approach: Test all HLA genotypes with expression to see if there is enrichement (fisher's exact test) <br>
Data: MHC-I/MHC-II typing; tapasin expression is from Su Xian

Tapasin expression groups

In [58]:
df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/clinical/TAPBP/TAPBP_expression.csv', header=None, sep='\t')
df.columns = ['last_four', 'TAPBP']

In [59]:
df.head()

,last_four,TAPBP
0,5630,1.559648
1,A7B0,6.077753
2,A829,3.320446
3,7222,4.938917
4,6972,2.883482


HLA genotypes

In [60]:
classI_types = pd.read_csv('/cellar/users/ramarty/Data/hla/hla_typing/sample_and_types.with_homomzygous.csv', index_col=0)
classI_types.index = list(classI_types['Sample'])
classII_types = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/hla_types/hla_types.tcga.csv', index_col=0)

In [61]:
classI_types = classI_types[['Sample', 'A1', 'A2', 'B1', 'B2', 'C1', 'C2']]

Restricting high/low tapasin expression to patients with types

In [62]:
restricted = classI_types[['Sample']]
def get_last_four_digits(x):
    return x[-4:]
restricted['last_four'] = classI_types['Sample'].apply(get_last_four_digits)

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [63]:
df = pd.merge(restricted, df, on='last_four', how='inner')

Add a column for number of alleles with each significant allele

In [64]:
results_df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/TAPBP/tapasin_hla_genotype.fisher_exact_results.csv',
                        index_col=0)

In [65]:
significant_alleles = list(results_df[:13].Allele)

In [66]:
for allele in significant_alleles:
    def allele_copy(x):
        occur = list(x).count(allele)
        return occur
    classI_types[allele] = classI_types[['A1', 'A2', 'B1', 'B2', 'C1', 'C2']].apply(allele_copy, axis=1)

In [67]:
expression_df = pd.merge(classI_types, df, on='Sample', how='inner')

In [68]:
expression_df.head()

,Sample,A1,A2,B1,B2,C1,C2,B*08:01,C*02:10,B*53:01,...,B*57:03,C*07:01,B*54:01,A*36:01,A*24:07,A*74:01,B*58:02,C*05:01,last_four,TAPBP
0,TCGA-P5-A780,A*01:01,A*02:01,B*07:02,B*35:03,C*04:01,C*07:02,0,0,0,...,0,0,0,0,0,0,0,0,A780,0.329949
1,TCGA-55-A494,A*02:01,A*24:02,B*15:01,B*57:01,C*03:03,C*07:01,0,0,0,...,0,1,0,0,0,0,0,0,A494,5.894547
2,TCGA-BR-6564,A*02:01,A*11:01,B*35:03,B*40:01,C*03:04,C*12:03,0,0,0,...,0,0,0,0,0,0,0,0,6564,5.953946
3,TCGA-BJ-A2P4,A*02:01,A*03:01,B*44:02,B*49:01,C*05:01,C*07:02,0,0,0,...,0,0,0,0,0,0,0,1,A2P4,0.211337
4,TCGA-N5-A4RJ,A*01:01,A*02:01,B*07:02,B*35:01,C*06:02,C*07:02,0,0,0,...,0,0,0,0,0,0,0,0,A4RJ,5.522056


In [69]:
expression_df.to_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/TAPBP/expression_df.csv')

In [70]:
for allele in significant_alleles:
    print classI_types[allele].value_counts()

0    8190
1    1526
2     123
Name: B*08:01, dtype: int64
0    9714
1     124
2       1
Name: C*02:10, dtype: int64
0    9616
1     204
2      19
Name: B*53:01, dtype: int64
0    9717
1     117
2       5
Name: B*15:03, dtype: int64
0    9387
1     432
2      20
Name: A*23:01, dtype: int64
0    9772
1      66
2       1
Name: B*57:03, dtype: int64
0    7263
1    2359
2     217
Name: C*07:01, dtype: int64
0    9803
1      34
2       2
Name: B*54:01, dtype: int64
0    9776
1      63
Name: A*36:01, dtype: int64
0    9811
1      25
2       3
Name: A*24:07, dtype: int64
0    9736
1      99
2       4
Name: A*74:01, dtype: int64
0    9769
1      69
2       1
Name: B*58:02, dtype: int64
0    8438
1    1321
2      80
Name: C*05:01, dtype: int64
